In [38]:
#  Change detect type
# input: nPosts

import difflib
import sys
import re
import nltk

In [83]:
d0 = '\
# Section0\n\
A example sentence that is embedded. The embedding is compared withanother\n\
# Section1'

# Edit of §0
d1 = '\
# Section0\n\
<del>A example sentence that is embedded. The embedding is compared withanother</del><ins>Faselbla</ins>\n\
# Section1'

# Edit of §1 (small)
d2 = '\
# Section0\n\
Faselbla\n\
# Section1\n\
<ins>wort wort wort wort</ins>\n'

# Edit of §1, triggers commit of POST d1
d3 = '\
# Section0\n\
Faselbla\n\
# Section1\n\
wort wort wort wort<ins>wort wort</ins>\n'

# Edit of §1 (small), triggers commit of POST d3
d4 = '\
# Section0<ins>adas</ins>\n\
Faselbla\n\
# Section1\n\
wort wort wort wort wort wort'

# Edit of §1
d5 = '\
# Section0<ins>wort wort wort wort</ins>\n\
Faselbla\n\
# Section1\n\
wort wort wort wort wort wort'

# Adding a paragraph
d6 = '\
# Section0\n\
Faselbla\n\
# Section1\n\
wort wort wort wort wort wort\n\
<ins> # Section2 Hallo I bims </ins>'

# Edit of §1 (small), triggers commit of POST d6
d7 = '\
# Section0<ins>adas</ins>\n\
Faselbla\n\
# Section1\n\
wort wort wort wort wort wort\n\
# Section2 Hallo I bims'

d = [d1, d2, d3, d4, d5, d6, d7]

In [87]:
t1 = '\
# The Black Hole\n\
A black hole is a region of spacetime exhibiting such strong gravitational effects that nothing—not even particles and <del>electrmoagnetic</del> <ins>electromagnetic</ins> radiation such as light—can escape from inside it. The <del>tehory</del> <ins>theory</ins> of general relativity predicts that a sufficiently compact mass can deform spacetime to form a black hole. (Source: wikipedia)\n\
\n\
# Inception\'s Depiction Of A Black Hole\n\
Kip Thorne is an astrophysicist; his math guided the creation of <del>the mesmareizing</del> <ins>this mesmerizing</ins> visual <del>effect in the movie Interstellar,</del> <ins>effect,</ins> the most <del>acurat</del> <ins>accurate</ins> simulation ever of what a black hole would look like. It\'s the product of a year of work by 30 people and thousands of computers. (Source: wired)'

t2 = '\
# The Black Hole\n\
A black hole is a region of spacetime exhibiting such strong gravitational effects that nothing—not even particles and electromagnetic radiation such as light—can escape from inside it. The theory of general relativity predicts that a sufficiently compact mass can deform spacetime to form a black hole. (Source: wikipedia)\n\
\n\
# Inception\'s Depiction Of A Black Hole\n\
Kip Thorne is an astrophysicist; his math guided the creation of this mesmerizing visual effect, the most accurate simulation ever of what a black hole would look like. It\'s the product of a year of work by 30 people and thousands of computers. (Source: wired)\n\
\n\
<ins># The First Picture Of A Black Hole</ins>\n\
<ins>Katie Bouman led development of a computer program that made the breakthrough image possible.</ins>\n\
<ins>The remarkable photo, showing a halo of dust and gas 500 million trillion km from Earth, was released on Wednesday. For Dr Bouman, its creation was the realization of an endeavor previously thought impossible. (Source: bbc)</ins>'


t3 = '\
# The Black Hole\n\
A black hole is a region of spacetime exhibiting such strong gravitational effects that nothing—not even particles and electromagnetic radiation such as light—can escape from inside it. The theory of general relativity predicts that a sufficiently compact mass can deform spacetime to form a black hole. (Source: wikipedia)\n\
\n\
# Inception\'s Depiction Of A Black Hole\n\
Kip Thorne <del>is an astrophysicist; his math guided the creation of this mesmerizing</del>  <ins>made some cool but wrong</ins> visual <del>effect, the most accurate simulation ever of what a black hole would look like. It\'s the product of a year of work by 30 people and thousands of computers. (Source: wired)</del> <ins>effects.</ins>\n\
\n\
# The First Picture Of A Black Hole\n\
Katie Bouman led development of a computer program that made the breakthrough image possible.\n\
The remarkable photo, showing a halo of dust and gas 500 million trillion km from Earth, was released on Wednesday. <del>For Dr Bouman, its creation was the realization of an endeavor previously thought impossible.</del> (Source: bbc)'

d = [t1, t2, t3]

In [40]:
def count_changes(difference):
    # Takes difference from wdiffhtml as input
    # Returns total number of changes in words
    dif = difference.splitlines()
    num_changes = 0
    for line in dif:
        if '<ins>' in line:
            word_list = nltk.word_tokenize(
                line[re.search('<ins>', line).end():re.search('</ins>', line).start()])
            num_changes += len(word_list)
        if '<del>' in line:
            word_list = nltk.word_tokenize(
                line[re.search('<del>', line).end():re.search('</del>', line).start()])
            num_changes += len(word_list)
    return num_changes

In [41]:
def which_paragraph_changed(text):
    list_of_paragraphs = re.findall(r"[^#]+", text)
    has_changes = [_check_if_has_diff(paragraph) for paragraph in list_of_paragraphs]
    #print(has_changes)
    return has_changes

def _check_if_has_diff(text):
    diff_tokens = ["<ins>", "<del>"]
    return any(diff_token in text for diff_token in diff_tokens)

In [106]:
paragraph_treshold = 5  # at least this many word changes have to be added/deleted in a paragraph to qualify for a paragraph commit
# POSTs without changes are discarded (assumption) so I assume every POST here has actual changes

scope_hist = []  # record scopes aka edited paragraphs of a POST
count_hist = []  # record count of words added/deleted by a POST
post_id_hist = []
paragraph_added_hist = []
posts_waiting = False  # are posts queued for a commit?
scope_switched = False  # did the scope switch since the last POST?
num_of_paragraphs = 0
for k, difference in enumerate(d):
    # Check scopes
    changes = which_paragraph_changed(difference)
    scope = np.arange(len(changes))[changes] if changes is not None else []
    #scope_hist.append(scope)
    scope_hist.append(tuple(scope))
    post_id_hist.append(k)
    single_scope = len(scope == 1)  # only POSTs that change a single Scope can be "§ edit" commits
    if len(scope_hist) > 1:
        scope_switched = (scope==scope_hist[-1])
        posts_waiting = True
    else:
        scope_switched = False
    
    count_hist.append(count_changes(difference))
    
    # Check if § was added TODO
    paragraph_added = (len(changes) - num_of_paragraphs) > 0 if k!=0 else False
    num_of_paragraphs = len(changes)
    paragraph_added_hist.append(paragraph_added)
    
    # Do commit
    # TODO COMMIT and delete corresponding histories
    # find out which POSTs to commit
    if single_scope:
        current_scope = scope
        words_changed = 0
        for i in reversed(range(len(scope_hist))):
            if all(scope_hist[i] != current_scope):#scope_hist[i] != current_scope:
                break
            else:
                words_changed += count_hist[i]
        print('POST ', k, ' added ',words_changed, ' words in §', scope[0])
    
    # Check if we qualified for a commit, then commit waiting POSTs (if there are POSTs waiting for commit)
    if posts_waiting and (words_changed >= paragraph_treshold): # then we made enough changes to qualify for a paragraph commit
        print('You should commit POST ', post_id_hist[i], ' as: ', end='')
        if any(paragraph_added_hist[:i+1]):
            print('Add a §')
        elif len(list(set(scope_hist[:i+1]))) > 1:
            print(scope_hist[:i+1], len(np.unique(scope_hist[:i+1])))
            print('small changes')
        else:
            print('Edit of § ', scope_hist[0][0])
        posts_waiting = False  # We commited all waiting POSTs in the queue
        
        # Clear POST history
        del(post_id_hist[:i+1])
        del(scope_hist[:i+1])
        del(count_hist[:i+1])
        del(paragraph_added_hist[:i+1])


POST  0  added  6  words in § 0
POST  1  added  67  words in § 1
You should commit POST  0  as: Edit of §  0
POST  2  added  100  words in § 1
You should commit POST  1  as: Add a §
